# 🏰 Escape Room Adventure Game V3

## 🌟 Características Mejoradas

- 🎨 Sistema visual mejorado con animaciones ASCII
- 🎵 Efectos de sonido
- 📦 Sistema de inventario
- 💡 Sistema de pistas
- ⏱️ Sistema de puntuación basado en tiempo
- 🏆 Sistema de logros
- 💾 Guardado/cargado de partida
- 🔄 Múltiples niveles de dificultad

In [1]:
# Instalación de dependencias necesarias
!pip install colorama playsound

import funcionesfinal_v2 as game_engine
from IPython.display import clear_output
import json
import time
from datetime import datetime

In [2]:
# Configuración del juego
GAME_CONFIG = {
    "difficulty": "normal",  # easy, normal, hard
    "sound_enabled": True,
    "hint_limit": 3,  # número de pistas disponibles
    "time_limit": 1800,  # 30 minutos en segundos
    "score_multiplier": {
        "easy": 0.5,
        "normal": 1.0,
        "hard": 2.0
    }
}

# Sistema de logros
ACHIEVEMENTS = {
    "speed_runner": {
        "name": "Speed Runner",
        "description": "Completa el juego en menos de 15 minutos",
        "unlocked": False
    },
    "treasure_hunter": {
        "name": "Treasure Hunter",
        "description": "Encuentra todos los tesoros",
        "unlocked": False
    },
    "master_explorer": {
        "name": "Master Explorer",
        "description": "Examina todos los objetos",
        "unlocked": False
    }
}

In [3]:
# Definición del mundo del juego
rooms = {
    "game_room": {
        "name": "game room",
        "type": "room",
        "description": "Una sala de juegos misteriosa iluminada por la tenue luz de velas antiguas. El piano antiguo emite un aura enigmática, y las sombras danzan en las paredes como si tuvieran vida propia."
    },
    "bedroom1": {
        "name": "bedroom1",
        "type": "room",
        "description": "Un dormitorio elegante con una cama queen size. Los tapices victorianos cubren las paredes, y un espejo ornamentado refleja la luz de manera extraña."
    },
    "bedroom2": {
        "name": "bedroom2",
        "type": "room",
        "description": "Un dormitorio más pequeño pero acogedor. Los muebles antiguos parecen guardar secretos de épocas pasadas, y un reloj de péndulo marca el tiempo con un ritmo hipnótico."
    },
    "livingroom": {
        "name": "livingroom",
        "type": "room",
        "description": "Una sala de estar majestuosa con una mesa de comedor intrigante. Las cortinas de terciopelo se mueven suavemente sin viento, y los cuadros en las paredes parecen seguir tus movimientos."
    },
    "outside": {
        "name": "outside",
        "type": "room",
        "description": "¡La libertad te espera! El aire fresco y la luz natural nunca se habían sentido tan reconfortantes."
    }
}

In [4]:
# Definición de muebles y objetos
furniture = {
    "couch": {
        "name": "couch",
        "type": "furniture",
        "description": "Un sofá vintage con patrones extraños que parecen cambiar cuando no los miras directamente.",
        "interaction": "Al sentarte, sientes una energía extraña recorrer tu cuerpo."
    },
    "queen_bed": {
        "name": "queen bed",
        "type": "furniture",
        "description": "Una cama queen size con sábanas de seda. Los patrones bordados forman símbolos misteriosos.",
        "interaction": "Las sábanas susurran secretos antiguos cuando las tocas."
    },
    "double_bed": {
        "name": "double bed",
        "type": "furniture",
        "description": "Una cama doble con un edredón que parece absorber la luz.",
        "interaction": "El colchón guarda memorias de sueños olvidados."
    },
    "dresser": {
        "name": "dresser",
        "type": "furniture",
        "description": "Un tocador antiguo con cajones que parecen tener vida propia.",
        "interaction": "Los cajones crujen con melodías olvidadas."
    },
    "dining_table": {
        "name": "dining table",
        "type": "furniture",
        "description": "Una mesa de comedor con marcas que forman un mapa estelar.",
        "interaction": "La superficie refleja constelaciones imposibles."
    },
    "piano": {
        "name": "piano",
        "type": "furniture",
        "description": "Un piano de cola que ocasionalmente toca notas por sí solo.",
        "interaction": "Las teclas responden a tu presencia con suaves melodías."
    }
}

In [5]:
# Definición de puertas y llaves
doors = {
    "door_a": {
        "name": "door a",
        "type": "door",
        "description": "Una puerta ornamentada con la letra 'A' que brilla con luz propia.",
        "mechanism": "Requiere una secuencia musical específica."
    },
    "door_b": {
        "name": "door b",
        "type": "door",
        "description": "Una puerta robusta marcada con 'B' que emite un zumbido bajo.",
        "mechanism": "Se abre con una combinación de presión y temperatura."
    },
    "door_c": {
        "name": "door c",
        "type": "door",
        "description": "Una puerta metálica con 'C' grabado que reacciona al tacto.",
        "mechanism": "Responde a patrones de luz específicos."
    },
    "door_d": {
        "name": "door d",
        "type": "door",
        "description": "La última puerta, marcada con 'D', vibra con energía antigua.",
        "mechanism": "Se abre solo cuando todos los secretos han sido descubiertos."
    }
}

keys = {
    "key_a": {
        "name": "key for door a",
        "type": "key",
        "target": doors["door_a"],
        "hint": "Una llave dorada con símbolos musicales que resuena con la frecuencia correcta.",
        "story": "Forjada por un antiguo maestro músico que creía en la magia de las melodías."
    },
    "key_b": {
        "name": "key for door b",
        "type": "key",
        "target": doors["door_b"],
        "hint": "Una llave plateada con patrones geométricos que cambian con la temperatura.",
        "story": "Creada por un alquimista que dominaba los elementos."
    },
    "key_c": {
        "name": "key for door c",
        "type": "key",
        "target": doors["door_c"],
        "hint": "Una llave de bronce con cristales que refractan la luz de manera única.",
        "story": "Diseñada por un inventor que estudió los secretos de la luz."
    },
    "key_d": {
        "name": "key for door d",
        "type": "key",
        "target": doors["door_d"],
        "hint": "Una llave antigua que solo funciona cuando todas las verdades son reveladas.",
        "story": "La última creación de un sabio que guardaba los secretos más profundos."
    }
}

In [6]:
# Definición de tesoros
treasures = {
    "ancient_book": {
        "name": "ancient book",
        "type": "treasure",
        "description": "Un libro antiguo que revela secretos olvidados.",
        "power": "Otorga conocimiento de lenguas antiguas.",
        "value": 1000
    },
    "crystal_orb": {
        "name": "crystal orb",
        "type": "treasure",
        "description": "Una esfera de cristal que muestra visiones del pasado.",
        "power": "Permite ver eventos históricos.",
        "value": 1500
    },
    "mystic_amulet": {
        "name": "mystic amulet",
        "type": "treasure",
        "description": "Un amuleto que pulsa con energía mística.",
        "power": "Aumenta la intuición del portador.",
        "value": 2000
    }
}

In [7]:
# Definición de relaciones entre objetos
object_relations = {
    # Contenido de las habitaciones
    "game room": [furniture["couch"], furniture["piano"], doors["door_a"]],
    "bedroom1": [furniture["queen_bed"], doors["door_a"], doors["door_b"], doors["door_c"]],
    "bedroom2": [furniture["double_bed"], furniture["dresser"], doors["door_b"]],
    "livingroom": [furniture["dining_table"], doors["door_c"], doors["door_d"]],
    "outside": [doors["door_d"]],
    
    # Ubicación de objetos
    "piano": [keys["key_a"], treasures["ancient_book"]],
    "queen bed": [keys["key_b"]],
    "double bed": [keys["key_c"], treasures["crystal_orb"]],
    "dresser": [keys["key_d"]],
    "dining table": [treasures["mystic_amulet"]],
    
    # Conexiones de puertas
    "door a": [rooms["game_room"], rooms["bedroom1"]],
    "door b": [rooms["bedroom1"], rooms["bedroom2"]],
    "door c": [rooms["livingroom"], rooms["bedroom1"]],
    "door d": [rooms["outside"]]
}

In [8]:
# Estado inicial del juego mejorado
INIT_GAME_STATE = {
    "current_room": rooms["game_room"],
    "keys_collected": [],
    "target_room": rooms["outside"],
    "treasure_collected": [],
    "escape_door": rooms["outside"],
    "inventory": [],
    "hints_remaining": GAME_CONFIG["hint_limit"],
    "start_time": time.time(),
    "time_limit": GAME_CONFIG["time_limit"],  # Agregado el tiempo límite
    "score": 0,
    "achievements": ACHIEVEMENTS.copy(),
    "examined_objects": set(),
    "difficulty": GAME_CONFIG["difficulty"]
}

# Iniciar el juego
clear_output(wait=True)
game_engine.start_game(INIT_GAME_STATE, object_relations)


    ╔═══════════════════════════════════════╗
    ║     ESCAPE ROOM - AVENTURA FINAL      ║
    ╚═══════════════════════════════════════╝
    
Despiertas en una mansión misteriosa, rodeado de enigmas y secretos.
No recuerdas cómo llegaste aquí, pero sientes que cada objeto guarda una historia.
Tu misión es escapar, pero ten cuidado: el tiempo corre en tu contra...


═══ COMANDOS DISPONIBLES ═══
explore: Explorar la habitación actual
examine <objeto>: Examinar un objeto específico
push <objeto>: Empujar un objeto
take <objeto>: Tomar un objeto
inventory: Ver inventario
use <objeto>: Usar un objeto del inventario
map: Mostrar el mapa
status: Mostrar estado actual
hint: Obtener una pista (limitado)
achievements: Ver logros
save: Guardar partida
load: Cargar partida
help: Mostrar esta ayuda
quit: Salir del juego

Estás en: game room

Exploras game room.
Una sala de juegos misteriosa iluminada por la tenue luz de velas antiguas. El piano antiguo emite un aura enigmática, y las sombras danz

KeyboardInterrupt: Interrupted by user

## 📖 Guía del Juego Mejorada

### 🎮 Comandos Disponibles
- `explore`: Explorar la habitación actual
- `examine <objeto>`: Examinar un objeto específico
- `push <objeto>`: Empujar un objeto
- `take <objeto>`: Tomar un objeto
- `use <objeto>`: Usar un objeto del inventario
- `inventory`: Ver tu inventario
- `map`: Mostrar el mapa del juego
- `status`: Ver tu progreso actual
- `hint`: Obtener una pista (limitado)
- `save`: Guardar partida
- `load`: Cargar partida
- `achievements`: Ver logros
- `help`: Mostrar esta ayuda
- `quit`: Salir del juego

### 🎯 Objetivos Principales
1. Encuentra las llaves ocultas en los muebles
2. Descubre los tesoros místicos
3. Resuelve los enigmas de cada habitación
4. Usa las llaves para abrir las puertas correctas
5. Escapa de la mansión antes de que se agote el tiempo

### 💡 Consejos Avanzados
- Cada objeto puede tener múltiples interacciones
- Los tesoros otorgan poderes especiales
- El tiempo afecta tu puntuación final
- Algunos secretos solo se revelan en ciertas condiciones
- Presta atención a los patrones y símbolos

### 🏆 Sistema de Logros
- Speed Runner: Completa el juego en menos de 15 minutos
- Treasure Hunter: Encuentra todos los tesoros
- Master Explorer: Examina todos los objetos

### 🗺️ Mapa Detallado
```
╔════════════════════════════════╗
║   [Game Room]─[Bedroom 1]─[Bedroom 2] ║
║           │         │              ║
║           └─[Living Room]─[Outside]║
╚════════════════════════════════╝
```

### 📊 Sistema de Puntuación
- Tiempo restante: +100 puntos por minuto
- Tesoros encontrados: +500-2000 puntos cada uno
- Logros desbloqueados: +1000 puntos cada uno
- Dificultad multiplicador:
  * Fácil: x0.5
  * Normal: x1.0
  * Difícil: x2.0